https://link.springer.com/book/10.1007/978-1-4612-3640-5 logic grammars
 https://link.springer.com/book/10.1007/978-3-642-83189-8  Foundations of Logic Programming
Prolog for AI: https://www.amazon.com/Programming-Artificial-Intelligence-International-Computer/dp/0321417461
The Curry tutorial: https://www.curry-lang.org/documentation/tutorial/
The power of prolog: https://www.metalevel.at/prolog

egraph that person sent


grpah rewriting chr

Fair interleaving

Stack lookup
do a condiuctive lp
meson pttp


https://hsd-pbsa.de/veranstaltung/scryer-prolog-meetup-2025/

typeclass system
rewrite system
conditional rewriter

Hmm. This is going to suck. For all the usual reasons.
Maybe reusing tptp stuff
maybe for knuckledragger is the wrong idea.
Or tag everything

https://github.com/turibe/swi-prolog-z3
scryer ffi to z3 c

saturate from henzinger


In [ ]:
@dataclass
class Scryer:
    decls : dict[str, smt.FuncDeclRef]

    def add(self, e : smt.ExprRef):
        decl = e.decl()
        name = decl.name()
        decl1 = self.decls.get(name)
        if decl1 is not None and decl1 != decl:
            raise ValueError(f"Duplicate declaration for {name}")
        self.decls[name] = decl
        if smt.is_const(e):
            return Term.Atom(e.decl().name())
        elif smt.is_app(e):
            return Term.Compound(e.decl().name(), tuple(to_scryer(arg) for arg in e.children()))
        


In [56]:
tptp.expr_to_tptp(e)

'(^[X__113:$int] : (f_f4 @ X__113))'

In [74]:
import kdrag.all
import kdrag.printers.tptp as tptp
import scryer
x = smt.Int("x")
f = smt.Function("f", smt.IntSort(), smt.IntSort())
machine = scryer.Machine()
e =  smt.Lambda([x], f(x))
prelude = r"""
:- op( 500, fy, !). % universal quantifier: ![X]:
:- op( 500, fy, ?). % existential quantifier: ?[X]:
:- op( 500, fy, ^).
:- op( 500, fy, $).
:- op( 500,xfy, :).
:- op( 500,xfy, @).

% --------------------

"""
s = tptp.expr_to_tptp(e)
print(s)
machine.load_module_string("mymodule",prelude +  f"formula({s}).")
machine.query_one("formula(X).")



(^[X__120:$int] : (f_f4 @ X__120))


RuntimeError: Prolog error term: error(existence_error(procedure, /(formula, 1)), /(formula, 1))

In [ ]:

def mangle(name):
    return name.replace("'", "_prime").replace("?", "_qmark").replace("!", "_emark")

def expr_to_prolog(e):
    if smt.is_const(e):
        return e.decl().name()
    elif smt.is_app(e):
        return f"{mangle(e.decl().name())}({', '.join(expr_to_prolog(arg) for arg in e.children())})"
    elif isinstance(e, smt.QuantifierRef):
        vs, body = kd.utils.open_binder(e)
        vs = 
        if e.is_forall():
            return f"forall([{', '.join(mangle(str(v)) for v in vs)}], {expr_to_prolog(body)})"
        elif e.is_exists():
            return f"exists([{', '.join(str(v) for v in vs)}], {expr_to_prolog(body)})"
        elif e.is_lambda():
            return f"lambda([{', '.join(str(v) for v in vs)}], {expr_to_prolog(body)})"
        else:
            raise ValueError(f"Unsupported expression type: {e}")
e = smt.Lambda([x], f(x))        
s = expr_to_prolog(e)
print(s)
machine.load_module_string("mymodule", f"formula({s}).")
machine.query_one("formula(X).")

lambda([X!112], f(X!112))


RuntimeError: Prolog error term: error(existence_error(procedure, /(formula, 1)), /(formula, 1))

In [32]:
from kdrag.all import *
import scryer
from scryer import Term

def to_scryer(e : smt.ExprRef,vs=[]) -> scryer.Term:
    vid = {v.get_id() for v in vs}
    if e.get_id() in vid:
        return Term.Var(f"V{e.get_id()}")
    if smt.is_const(e):
        return Term.Atom(e.decl().name())
    elif smt.is_app(e):
        return Term.Compound(e.decl().name(), tuple(to_scryer(arg) for arg in e.children()))
    else:
        raise ValueError("Unsupported SMT expression", e)
T = smt.DeclareSort("Term")
def from_scryer(t : scryer.Term) -> smt.ExprRef:
    match t:
        case scryer.Term.Atom(name):
            return smt.Const(name, T)
        case scryer.Term.Compound(name, args):
            return smt.Function(name, *[T]*len(args), T)(*[from_scryer(arg) for arg in args])
        case scryer.Term.Var(name):
            c = smt.Const(name, T)
            return c
        case _:
            raise ValueError("Unsupported Scryer term", t)

    
x,y,z = smt.Ints('x y z')
to_scryer(x + y)

machine = scryer.Machine()
machine.load_module_string("mymod", "id(X,X).")
res = machine.query_one(f"id({str(to_scryer(x + y))}, Y).")
from_scryer(res["Y"])


+(x, y)

In [26]:
%%file /tmp/play.pl
:- use_module(library(lists)).
%rw(A + B, B + A).
rw(a,b).
rw(+(a,b), +(b,a)).
rw(X + 0, X).
rw(0 + X, X).


simplify_bu(T, S) :-
    ( compound(T)
    -> T =.. [F|Args],
       maplist(simplify_bu, Args, ArgsS),
       T1 =.. [F|ArgsS],
       ( rw(T1, T2) -> simplify_bu(T2, S) ; S = T1 )
    ;  S = T
    ).



Overwriting /tmp/play.pl


In [28]:
! scryer-prolog --goal "rw(a, Y), write(Y), rw(a + b, Z), write(Z), simplify_bu(x + 0 + 0 + 0, Q), write(Q), halt." /tmp/play.pl

bb+ax

In [54]:
import scryer
machine = scryer.Machine()
machine.query_all("Y = [ a | X], X = [ba,d], Z = .(a, [b,d]).")

[{'Z': String { value: "abd" },
  'X': List { values: [Atom { value: "ba" }, Atom { value: "d" }] },
  'Y': List { values: [Atom { value: "a" }, Atom { value: "ba" }, Atom { value: "d" }] }}]

In [38]:
1 + 1

2

In [15]:
machine.load_module_filename("mymod","/tmp/test.pl")

In [19]:
machine.query_all(f"hello(b).")

[]

In [22]:
world = machine.query_one("hello(X).")["X"]
machine.query_one(f"hello({world}).")
machine.query_one("Z = [a, b | X]")


thread '<unnamed>' (494361) panicked at /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/scryer-prolog-0.10.0/src/machine/lib_machine/mod.rs:572:14:
Failed to parse query: IncompleteReduction(0, 12)


PanicException: Failed to parse query: IncompleteReduction(0, 12)

https://www.scryer.pl/

In [2]:
%%file /tmp/test.pl

hello(world).


Writing /tmp/test.pl


In [3]:
! scryer-prolog /tmp/test.pl -g 'hello(X), write(X), halt.'

world

In [4]:
! swipl -g "hello(X), write(X), halt." /tmp/test.pl

world

## Why Prolog?
https://news.ycombinator.com/item?id=31210466
https://github.com/mitsuchi/copl-in-prolog/
https://github.com/hsk/simpletapl/blob/master/prolog5/fullupdate.pl
Norvig
Bratko
Art of prolog https://cliplab.org/~logalg/doc/The_Art_of_Prolog.pdf
Power of prolog  https://www.metalevel.at/prolog
Clocksin and Mellish
lambda prolog examples (?)
minikanren stuff

Type checkers
evaluation semantics

Typeclass resolution.

Leantap
Leancop
nanocop
PTTP


term rewriting, e-unification
KB


I don't find sudokus compelling. I think you're better off with other techniques.

# CLP
freeze(X,goals)
-->
state has mapping Var -> suspgoals

CLP(set) using CLP(B) keys trie
or sorted lists of CLP(Z). disjoint union. separation.

CLP vs eager e-unify


In [ ]:
from typing import NamedTuple

class State(NamedTuple):
    vs  : list[smt.ExprRef]
    goals : list[smt.BoolRef]
    subst : dict[smt.ExprRef, smt.ExprRef]
    constrs : list[smt.BoolRef]

    def is_sat(self) -> bool:
        s = smt.Solver()
        s.add(*self.goals)
        s.add(*self.constrs)
        s.add(*[k == v for k, v in self.subst.items()])
        return s.check() == smt.sat
    def propagate(self):
        s = smt.Solver()
        s.add(*self.goals)
        s.add(*self.constrs)
        s.add(*[k == v for k, v in self.subst.items()])
        if s.check() == smt.sat:
            m = s.model()
            for d in m.consts():
                with s:
                    s.add(d == m.eval(d))
                    if s.check() == smt.unsat:
                        #do substitution 
                        lhs = d()
                        rhs = m[d]
                        if lhs in self.subst:
                            assert self.subst[lhs] == rhs
                        self.subst[lhs] = rhs
        else:
            return None
    def subst_add(self, x, t):
        new_vs = [v for v in self.vs if not v.eq(x)]
        assert len(new_vs) + 1 == len(self.vs)
        new_subst = {**{v: smt.substitute(e, (x,t)) for v,e in self.subst.items()}, x: t}
        new_goals = [smt.substitute(g, (x, t)) for g in self.goals]
        new_constrs = [smt.substitute(c, (x, t)) for c in self.constrs]
        return State(new_vs, new_goals, new_subst, new_constrs)


def horn(prog, goal):
    states : list[State] = [] # fifo, lifo, other
    while states:
        state = states.pop()
        if state.is_sat() is False:
            continue
        if not state.goals:
            yield state
            continue
        else:
            goal = state.goals.pop()
            if smt.is_and(goal):
                state.goals.extend(smt.children(goal))
                states.append(state)
                continue
            elif smt.is_or(goal):
                for disj in smt.children(goal):
                    new_state = State(state.goals + [disj], state.subst.copy(), state.constrs.copy())
                    states.append(new_state)
                continue
            elif goal.decl() in clp_preds:
                    # constraint logic predicate
                    state.constrs.append(goal)
                    states.append(state)
                    continue
                for prog in progs:
                    # unify
                    # do subst into everything



eunify

In [ ]:
decls = {
    
}

## CLP(Egraph)
Use CHR union find. Local state. Why didn't I use chr here?
https://www.philipzucker.com/harrop-checkpoint/

```
impl(eq(A,B),G) :- chr_add(A,B), G.  
unify_egraph(T1,T2) :- T1 = T2.
unify_egraph(T1,T2) :- find(T1) = find(T2). # ematch(T1), ematch(T1).

```


# Lambda
https://github.com/LPCIC/elpi

In [15]:
%%file /tmp/lam.elpi
pred a o:int.
a 1.

main :- a X, print X.

Overwriting /tmp/lam.elpi


In [16]:
! elpi -test /tmp/lam.elpi


Parsing time: 0.000

Compilation time: 0.000

Typechecking time: 0.004
Determinacy checking time: 0.004
1

Success:

Time: 0.000

Constraints:


State:



# Coinductive and PTTP
Stack inspection is key to coinductive logic programming, PTTP

# FLP